In [35]:
import json

with open("distinct_models.json") as f:
    distinct_models = json.load(f)

with open("distinct_descriptions.json") as f:
    distinct_descriptions = json.load(f)

with open("test.json") as f:
    all_cars = json.load(f)

from_all = set([c["build"]["model"] for c in all_cars])
from_all

{'A-Class',
 'AMG GT',
 'AMG GT 4-Door Coupe',
 'AMG GT Coupe',
 'AMG GT Roadster',
 'B-Class',
 'C-Class',
 'C-Class Cabriolet',
 'C-Class Coupe',
 'C-Class Sedan',
 'CL-Class',
 'CLA',
 'CLA-Class',
 'CLE',
 'CLK-Class',
 'CLS-Class',
 'E-Class',
 'EQB',
 'EQE',
 'EQE SUV',
 'EQS',
 'EQS SUV',
 'G-Class',
 'GL-Class',
 'GLA',
 'GLA-Class',
 'GLB',
 'GLC',
 'GLC Coupe',
 'GLC-Class',
 'GLE',
 'GLE Coupe',
 'GLE-Class',
 'GLE-Class Coupe',
 'GLK-Class',
 'GLS',
 'GLS-Class',
 'M-Class',
 'Metris Cargo Van',
 'Metris Passenger Van',
 'S-Class',
 'SL',
 'SL Roadster',
 'SL-Class',
 'SLC Roadster',
 'SLK',
 'SLK-Class',
 'Sprinter Cab Chassis',
 'Sprinter Cargo Van',
 'Sprinter Crew Van',
 'Sprinter Passenger Van',
 'Sprinter Van'}

In [36]:
import json
from backend.db.products import product_db, Product, VehicleType, Powertrain, Color
from pprint import pprint
import traceback


# Problem with normalization of Mini MPV
def normalize(value):
    return "Mini MPV" if value == "Mini Mpv" else value


product_list = []
for model in from_all:
    for car in all_cars:
        if car["build"]["model"] == model:
            try:
                desc = None
                for d in distinct_descriptions:
                    if d["model"] == model:
                        desc = d["description"]
                        print(f"desc={desc}")
                        break
                else:
                    print("no description found")
                    desc = "No description available"

                product = Product(
                    name=car["heading"],
                    description=desc,
                    vehicle_type=VehicleType(normalize(car["build"]["body_type"])),
                    release_year=car["build"]["year"],
                    price_in_usd=car.get("price")
                    or car.get("ref_price")
                    or car.get("msrp"),
                    is_used=car["inventory_type"] == "used",
                    powertrain_type=Powertrain[
                        "EV"
                        if car["build"].get("powertrain_type") == "BEV"
                        else "ICE"
                        if car["build"].get("powertrain_type") == "Combustion"
                        else "Hybrid"
                    ],
                    num_seats=car["build"]["std_seating"],
                    color=car.get("base_ext_color"),
                    specs=car["build"],
                    image_url=car["media"]["photo_links"][:1]
                    if car.get("media")
                    else [],
                )

                # override image from distinct_models
                for m in distinct_models:
                    if m["model"] == model:
                        better_img = m["links"][:1]
                        if better_img:
                            print("overriding image")
                            product.image_url = better_img
                            break

                product_list.append(product)
                break

            except Exception as e:
                # print("skipping car")
                # traceback.print_exc()
                # print("Error in parsing the following listing:")
                # pprint(car)
                continue

desc=The Mercedes-Benz CLA is a luxury compact executive car introduced by Mercedes-Benz in 2013 as a stylish, sporty four-door coupe. Positioned above the A-Class and near the C-Class in the Mercedes lineup, the CLA has been celebrated for combining elegance, sportiness, and performance. The first generation was based on the platform of the A-Class and B-Class, making its debut in Europe in April 2013 and in the U.S. in September 2013. It features models like the high-performance CLA 45 AMG, known for its dynamic driving experience, and a special edition CLA Edition 1. In 2019, the second generation was launched, offering updated styling, advanced technologies like the MBUX infotainment system, and a new CLA Shooting Brake with additional space. The 2023 update saw aesthetic and functional improvements, including a redesigned exterior, new interior options, enhanced safety features, and varied engine options including mild-hybrid and plug-in hybrids, highlighting its evolution into a 

In [37]:
len(product_list)

51

In [42]:
# display the images in jupyter
from IPython.display import Image, display

for product in product_list:
    print(product.image_url)
    print(product.description)
    display(Image(url=product.image_url[0]))

['https://assets.oneweb.mercedes-benz.com/iris/iris.png?COSY-EU-100-1713d0VXqaSFqtyO67PobzIr3eWsrrCsdRRzwQZUkwZbMw3SGtle9tsd2HtcUfp8qXGEunSJ0l3DAOB2NbFbApRARI5ux6YQC30M1kzNBzkm7jAymhKV5XN%25vq4t9yLRgLyYaxPadrH1eH6n8ws8WoiZUkEM4FGTJTg9oQZ6PDMLoSeWza3tsd7ktcUfi%25fXGE4bjJ0lg6JOB2PSFbApeIoI5usKxQC3UvWkzNGoum7juyIhKVHtc%25YhQDd95MXGEiGRJ0l846OB2GM1bAp0ToI5uC5JQC3zgOkzN7U9m7jKyZhKVvXt%25vqLukyL0fsjko9wdYf0hzjtCRcYxqN88Zlo21Ix2rbpldCdcqcgVGB4NPVHF82dQ=&BKGND=9&IMGT=P27&POV=BE040%2CPZM&im=Trim&fuzz=0.5&width=670']
The Mercedes-Benz CLA is a luxury compact executive car introduced by Mercedes-Benz in 2013 as a stylish, sporty four-door coupe. Positioned above the A-Class and near the C-Class in the Mercedes lineup, the CLA has been celebrated for combining elegance, sportiness, and performance. The first generation was based on the platform of the A-Class and B-Class, making its debut in Europe in April 2013 and in the U.S. in September 2013. It features models like the high-performance CLA 45

['https://assets.oneweb.mercedes-benz.com/iris/iris.png?COSY-EU-100-1713d0VXqaJeqtyO67PobzIr3eWsrrCsdRRzwQZ63pZbMw3SGtxX2tsd1ZpcUfwMuXGEuymJ0l3YZOB2NrjbApjiVI5uVeuQC3qsFkzNR%25nm7jxyIhKV1Yh%25vqwJkyLRZOVYaxFkErH19nun8wDqfoiZWJXM4FlIDTg92QE6PDpPpSeWue9tsd3s%25cUfNLkXGEjaRJ0lVHNOB2qQ1bApRTyI5ux6ZQC31EFkzNwldm7jA6ohKV5Kh%25vqCUTyLRzGFYax759rH1KM1n8wvVwoiZL%25pM4FaTQTg90Q26PDBkbSeWAeStsd5spcUfCLqXGEziwJ0l7OhOB2KbFbApJilI5uOcxQC3iMwkzN4Pkm7jgcfhKVPX9%25vqeIkyLRsQHYaxU4drH1Gcrn8w7OwoiZKiEM4FvSlTg9Lte6PDa%25bSeWHw9tsd8oGcUfiXyXGE45jJ0lgChOB2PzObApe7pI5usKDQC3UnpkzNpTwm7ju6ohKV3YQ%25vq8tTyLRiO6Yax4bJrH1gjrn8wPbIoTnylpEvcQC3sE3kzNetDm7jC7fhKVzst%25vq7BTyLRKn2YaxvoJrH1LIln8waIIoiZHQUM4F8QETg9iGm6PDVkbSeWqiftsdUhQcUfGUNXGE0GYJ0lBHtOB2A81bAp5iCI5uC4MQC3zs3kzN7U9m7sDgubYwR9hDvVCBdwj6hVNpLLxdYfqJVf%25XEd9B96N683eUHpi3v1Lf9O=&BKGND=9&IMGT=P27&POV=BE040%2CPZM&im=Trim&fuzz=0.5&width=670']
The Mercedes GLC-Class, developed by Mercedes-Benz, is a versatile and luxurious SUV known for its combination of s

['https://assets.oneweb.mercedes-benz.com/iris/iris.png?COSY-EU-100-1713d0VXqaW7qtyO67PobzIr3eWsrrCsdRRzwQZ6TfZbMw3SGtle9tsd2HdcUfp8qXGEubmJ0l3ItOB2NQnbApjtwI5uVQDQC31MjkzNwzwm7jZeohKVFXM%25vqCvqyLRz3RYaxPXWrH1eHDn8wsQfoiZUMEM4FGTjTg9ovn6PDC%25qSeWzy3tsd8hdcUfiONXGE4nXJ0lgDfOB2PSObApeIyI5usQMQC3UvrkzNGLdm7j0M3hKVHtc%25YhQDd95MXGEiydJ0l846OB2GB2bAp0AwI5u9QzQC3zCOkzN7Pbm7jKeDhKUWP3IrZxD%25WLNhcRZVS%25qjuaa1frEROqEyJlfDADSjSiNsG8u4NLwaEDb=&BKGND=9&IMGT=P27&POV=BE040%2CPZM&im=Trim&fuzz=0.5&width=670']
The Mercedes C-Class Sedan epitomizes luxury and high performance in a compact executive car format, first introduced in 1993. Across its various generations, including the most recent W206 launched in 2021, it offers a blend of advanced technology, stylish design, and refined driving dynamics. Key highlights include a range of powerful four-cylinder engines equipped with an integrated starter generator and a 48-volt electrical system. The latest model, the Mercedes-AMG C 63 S E PERFORMANCE, 

['https://assets.adac.de/image/upload/w_898/Autodatenbank/GWInfo/gw0303-mercedes-benz-slk-klasse-2004-2011.jpg']
The Mercedes-Benz SLK, later known as the SLC from 2016 onwards, is a compact executive roadster that has been celebrated for its sporty design and innovative features. First introduced in 1996, the SLK-Class quickly gained recognition for its distinctive retractable hardtop called the "Vario-Dach," which allowed it to switch between a hardtop and a convertible. The SLK spans three generations, with various upgrades including enhancements in engine performance ranging from four-cylinder to V8 options, as well as both diesel and petrol engines. Its notable features include Magic Sky Control, an electronically operated tint shade for the sunroof, and a progressive aesthetic update in 2016 to commemorate its 20th anniversary. The SLK has been lauded for its synthesis of luxury, sportiness, and innovative technology, making it a well-rounded vehicle ideal for both performance en

['https://assets.oneweb.mercedes-benz.com/iris/iris.png?COSY-EU-100-1713d0VXqrWFqtyO67PobzIr3eWsrrCsdRRzwQZgk4ZbMw3SGtxX2tsd1ZpcUfwMuXGEuGdJ0l34lOB2Ng1bApjkbI5uVmMQC3qSpkzNRJ9m7jxaShKV1TM%25vq9tcyLRDLVYaxW0SrH1dItn8wfV2oiZEqNM4FaIMTg9HzO6PD8LFSeWiwWtsd4oVcUfgX0XGEzyjJ0l70JOB2KBjbApvPpI5uLeDQC3aSrkzNHmnm7jQe3hKVPS9%25vqeUDyLRsnDYaxU4drH1zJ1n8w7VxoiZK%25XM4FvyMTg9L6O6PDaSmSeWH0Rtsd8kxcUfimWXGEWbSJ0lCYhOB2zr5bAp7TyI5uKMTQmIJwF1GY6PDLSpSeWvw5tsdPJdcUfeOyXGEs9EJ0lweROB2Ar%25bAp5ARI5uCeZQC3zhFkzN7lwm7jKh3hKUWP3IrZxD%25WLcZnfAVS%25qjuaa1frEROqEyJlfDADSjSiNsG8u4NLwaEDb=&BKGND=9&IMGT=P27&POV=BE040%2CPZM&im=Trim&fuzz=0.5&width=670']
The Mercedes SL-Class, a prestigious grand touring sports car produced by Mercedes-Benz since 1954, embodies a blend of luxury, performance, and advanced technology. Originally launched with the historical 300 SL model, the SL-Class has evolved through various iterations, including model ranges like the W198, W121, R107, among others, each marking significant advance

...

['https://assets.oneweb.mercedes-benz.com/iris/iris.png?COSY-EU-100-1713d0VXqaWFqtyO67PobzIr3eWsrrCsdRRzwQZYZ4ZbMw3SGtGyWtsd2sDcUfp8fXGEuiRJ0l3IJOB2NMcbApjTlI5ux5xQC31SrkzNBzkm7jAc3hKViSF%25vq4v%25yLRgOjYaxPr9rH1eOdn8wYAxoiZr6YM4FAClTg95vp6PDCIoSeWHmWtsd8J3cUfiMcXGE4TwJ0lgOrOB2PzqbApe79I5usr2QC32hOkzNLzkm7jaSthymI9WFAocUfa8MXGEsnRJ0lUorOB2ZgEbAp5AXI5uC4JQC3zh1kzN7t6m7jK2IhKUWP3IrZxD%25WLfsQ6uVS%25qjuaa1frEROqEyJlfDADSjSiNsG8u4NLwaEDb=&BKGND=9&IMGT=P27&POV=BE040%2CPZM&im=Trim&fuzz=0.5&width=670']
The Mercedes-Benz GL-Class is a luxury full-size SUV that debuted in 2006 and has become known for its spacious accommodation, advanced technology, and commanding performance. This SUV can seat up to seven passengers and combines the utility of a family vehicle with the elegance of a high-end Mercedes-Benz. The GL-Class features various engine choices from the efficient GL 320 CDI with a six-cylinder engine producing 224 horsepower, up to the formidable GL 500 equipped with a V8 engine deliveri

['https://assets.oneweb.mercedes-benz.com/iris/iris.png?COSY-EU-100-1713d0VXqNWFqtyO67PobzIr3eWsrrCsdRRzwQZUkRZbMw3SGtGyStsd2HtcUfp8qXGEubYJ0l36xOB2qrnbApRn3I5ux6YQC30C3kzNBUbm7jAGvhKV5YL%25vq4t9yLRgc6YaxPaYrH1eHDn8wsQfoiZUMXM4FnCwTg9otn6PDCLsSeWzaRtsd7oTcUfKM0XGE4yXJ0lg0VOB2PQqbApedlI5usQYQC3UkTkzNGLnm7j0aShKVBTN%25vqNtkyLRiXJYnymdEWz6OB2iMwbAp0iyI5uB4MQC3AhTkzNW5Jm7jK7ShKVvsL%25vqLUcyLRaifYaBEUVmMDZfrE810zyRxXr1RjiiF2MpwQ1pnIu2fzfXRXPq0Agjeq89ipfk=&BKGND=9&IMGT=P27&POV=BE040%2CPZM&im=Trim&fuzz=0.5&width=670']
It appears I'm facing an issue with fetching the online data right now. However, I can give you a general idea based on my last update.

The Mercedes CLS-Class is a series of luxury sedans renowned for its sleek, coupe-like silhouette that blends elegance with aggressive styling. Introduced by Mercedes-Benz, it combines high-performance engineering with upscale amenities and cutting-edge technology. The interior of the CLS-Class typically features luxurious materials, advanced i

['https://assets.adac.de/image/upload/w_898/Autodatenbank/GWInfo/gw0303-mercedes-benz-slk-klasse-2004-2011.jpg']
The Mercedes-Benz SLK-Class, produced between 1996 and 2020, is a series of luxury compact executive roadsters renowned for its sophisticated retractable hardtop roof known as the "Vario-Dach." Developed in Bremen, Germany, the SLK-Class comprises three generations: R170, R171, and R172, each progressively incorporating enhancements in design, technology, and performance. Notable features across models include various engine options such as V6 and V8, the innovative Magic Sky Control for electronically tinted glass, and advanced safety technologies. Positioned as a compact and sporty option within the luxury segment, the SLK-Class boasts a blend of performance, innovation, and dynamic driving that appeals equally to style-conscious and enthusiast drivers alike.


['https://assets.oneweb.mercedes-benz.com/iris/iris.png?COSY-EU-100-1713d0VXqaSFqtyO67PobzIr3eWsrrCsdRRzwQZgk4ZbMw3SGtlaStsd2HVcUfpOkXGEubSJ0l3IrOB2NS1bApj7pI5uVKMQC31MTkzNwPnm7jZcohKV5Kh%25vqCr%25yLRz3RYaxPXWrH1eJtn8ws8WoiZUMXM4FGTjTg90kO6PDBIDSeWTwItsd7kTcUfi%25fXGE4GjJ0lgIVOB2PWEbApetXI5usc2QC3UkrkzNGmbm7j0hShKVBHM%25vqA8lyLR5SVYax4bSrH1gObnMr%25E2fKtI5ue6YQC3PsXkzN5z9m7jd2ohKVvKQ%25vqLUkyLRaGfYaxHoJrHAlGqhTWFEnlioUiXW1JnwxV449pTuZkwuoQ3pE7EJxJeRB5PVsRiD4uEm=&BKGND=9&IMGT=P27&POV=BE040%2CPZM&im=Trim&fuzz=0.5&width=670']
The Mercedes EQE is a luxury electric sedan that combines advanced technology, comfort, and performance. It is positioned between the E-Class and the flagship EQS, featuring a slightly smaller size compared to the EQS but similar design and technology elements. The EQE boasts a spacious interior with high-quality materials and provides a quiet ride thanks to effective noise insulation. Key features include the MBUX Hyperscreen, which spans across the entire dashboard

['https://assets.oneweb.mercedes-benz.com/iris/iris.png?COSY-EU-100-1713d0VXqbE7qtyO67PobzIr3eWsrrCsdRRzwQZYkpZbMw3SGtGyStsd2vtcUfpLfXGEuiXJ0l34AOB2NQnbApjtwI5uVczQC3qkOkzNRLkm7jxafhKVFSQ%25vq9tayLRDLVYaxW0SrH1dB%25n8wfA2oiZEkXM4FlTlTg9HQO6PD8koSeWimUtsd4stcUfg86XGEPiYJ0leIJOB2sQ1bApUk9I5uG6YQC30SOkzNBtkm7j86ohKViSh%25vq4r9yLRgnyYaxPNSrH1ej%25n8wsoyoiZUMNM4FGOPTg9ohE6PDC7sSeWzK3tsd7HVcUfKMcXGEvTVJ0lLOqOB2PrObApeAyI5usmxQC3US3kzNGlum7j0OjhKVBbN%25vqAyTyLR5YXYaxC4SrH1zgRn8w7XxoiZxbgM4F1IjTg9wtn6PDGmhSc6o3ju8b%25vqBBjyLR0Wm7tBodhKkt3f%25vm1fEyLhCPlYaB6z2rHn15pn8onZuoiMIT3M4Th5NTg6jij6hzQipxBUCVzt9CtP8PDVeg8ccGKxvslevqELKCrCDgDO4m%25J8b4t0cvC2=&BKGND=9&IMGT=P27&POV=BE040%2CPZM&im=Trim&fuzz=0.5&width=670']
The Mercedes S-Class, recognized as the epitome of luxury in automotive engineering, is a flagship luxury sedan from the iconic German automaker, Mercedes-Benz. Introduced in its modern form in 1972, the S-Class has continuously set benchmarks in technology, safety, and performance across

In [39]:
# save to db
await product_db.reset()
await product_db.add(product_list)

In [40]:
await product_db.size()

51